In [1]:
!pip install lxml

In [4]:
from lxml import etree

ImportError: dlopen(/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lxml/etree.cpython-39-darwin.so, 0x0002): tried: '/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lxml/etree.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))

In [2]:
# Libraries that are used in the program
import os
os.chdir(os.path.join("src", "cbr"))

In [3]:
import random
import pickle

# Own libraries
from cbr import CBR
from entity.query import Query
from definitions import CASE_LIBRARY as CASE_LIBRARY_PATH, DATA_PATH
from case_library import CaseLibrary, ConstraintsBuilder

random.seed(10)

ImportError: dlopen(/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lxml/objectify.cpython-39-darwin.so, 0x0002): tried: '/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lxml/objectify.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))


# CBR IMPLEMENTATION

Steps:


1. Retrive the data from the case Libray
2. Retrieve the new case
3. Compute the similarity between the new case and the existing cases
4. Retrieve the existing case with the highest similarity
6. Select the best case and rule
7. Adapt the solution
8. Update the database
9. Evalute the solution
10. If the solution is not good, repeat the process
11. If the solution is good, stop the process and return the solution and save it in the case Library
12. Retrieve the new case

In [3]:
# Pickle file containing the data
path = os.path.join(DATA_PATH, "processed-cocktails-data.pkl")
# Load the data <data>
cocktails = pickle.load(open(path, "rb"))

In [4]:
print(f"The number of instances from the database {len(cocktails)}")
cocktails.head()

The number of instances from the database 1781


,Cocktail,Category,Glass,Ingredient,Alc_type,Basic_taste,Steps,Measure,Value_ml,Value_gr,Garnish_amount,Garnish_type
index,,,,,,,,,,,,
0,'57 Chevy with a White License Plate,cocktail,highball glass,creme de cacao white,creamy liqueur,NaN,1. fill a rocks glass with ice 2.add white cre...,1 oz white,30.0,0.0,0.0,NaN
1,'57 Chevy with a White License Plate,cocktail,highball glass,vodka,vodka,NaN,1. fill a rocks glass with ice 2.add white cre...,1 oz,30.0,0.0,0.0,NaN
2,1-900-FUK-MEUP,shot,old-fashioned glass,grand marnier,triple sec,NaN,shake ingredients in a mixing tin filled with ...,1/4 oz,7.5,0.0,0.0,NaN
3,1-900-FUK-MEUP,shot,old-fashioned glass,midori melon liqueur,sweet liqueur,NaN,shake ingredients in a mixing tin filled with ...,1/4 oz,7.5,0.0,0.0,NaN
4,1-900-FUK-MEUP,shot,old-fashioned glass,malibu rum,rum,NaN,shake ingredients in a mixing tin filled with ...,1/4 oz,7.5,0.0,0.0,NaN


## RETRIEVE

In [3]:
query = Query()
query.set_category("ordinary drink")
query.set_glass("old-fashioned glass")
query.set_ingredients(["orange juice"])
query.set_exc_ingredients(["rum", "grenadine"])
query.set_alc_types(["gin"])
query.set_basic_tastes(["salty"])

# TODO: retrieve 5 recipes similar to the query from the case library
# Random recipes for now
CASE_LIBRARY = CaseLibrary(CASE_LIBRARY_PATH)
CONSTRAINT = ConstraintsBuilder(include_category=query.get_category(), include_glass=query.get_glass())
recipes = random.choices(CASE_LIBRARY.findall(CONSTRAINT), k=5)

## ADAPT

In [4]:
print(f"Ingredients before: {[e.text for e in recipes[0].ingredients.iterchildren()]}")
print(f"Steps before: {[e.text for e in recipes[0].preparation.iterchildren()]}")

cbr = CBR(query, recipes)
cbr.adapt()
output = cbr.recipe

print(f"Ingredients after: {[e.text for e in output.ingredients.iterchildren()]}")
print(f"Steps after: {[e.text for e in output.preparation.iterchildren()]}")

Ingredients before: ['151 proof rum', 'grenadine', 'kahlua']
Steps before: ['mix ingr2 and ingr0 in glass', 'quickly add ice and pour ingr1 over ice to give ice red tint']
Ingredients after: ['151 proof rum', 'orange juice', 'kahlua', 'gin', 'salt']
Steps after: ['mix ingr2 and ingr0 in glass', 'add ingr4 to taste', 'add 2 oz of ingr3', 'quickly add ice and pour ingr1 over ice to give ice red tint']


## EVAL

## LEARN